# Graphene Sheet with Interactive Physics Parameters

This notebook simulates a graphene sheet with OpenMM via ASE, and exposes parameters that can be controlled in real time.
Along the way, we'll learn:

* How to add restraints to OpenMM simulations 
* How to expose functions as **commands** that can be run remotely 
* How to add sliders and buttons to Jupyter notebooks

## Setup the OpenMM Simulation 

First, we set up an OpenMM simulation of graphene. In this case, we've already generated an OpenMM XML file and have a PDB ready to use for the topology. See the [neuraminidase example](./ase_openmm_neuraminidase.ipynb) for a more detailed look at setting up OpenMM simulations. 

In [1]:
import openmm.app as app
import openmm as mm 
import openmm.unit as unit
pdb_file = app.PDBFile('openmm_files/graphene_with_bonds.pdb')
system_xml = 'openmm_files/graphene_omm.xml'

In [2]:
#read the system into OpenMM 
with open(system_xml, 'r') as f:
    system_string = f.read()
system: mm.System
system = mm.XmlSerializer.deserialize(system_string)

Great, we've got a PDB topology, and a definition of all of the OpenMM forces to use with it. 

In [3]:
system.getNumForces(), system.getNumParticles()

(3, 680)

In [4]:
pdb_file.getTopology().getNumAtoms()

680

For this simulation, we want to hold the corners of the graphene sheet in place. We do that with spring force restraints. 
NanoVer has some shortcuts for setting these up with OpenMM.

In [5]:
from nanover.openmm.potentials import restrain_particles

In [6]:
atoms_to_restrain = [0, 38, 641, 679] # the corner atoms. 
force = restrain_particles(pdb_file.positions, atoms_to_restrain, 10000 * unit.kilojoule_per_mole / unit.nanometer ** 2)
force_index = system.addForce(force)
print(f"Added force with index {force_index}. System now has {system.getNumForces()} forces.")

Added force with index 3. System now has 4 forces.


Now we create an OpenMM simulation with it. 

In [7]:
simulation = app.Simulation(pdb_file.topology, system, mm.LangevinIntegrator(300 * unit.kelvin, 1.0/unit.picosecond, 1.0*unit.femtosecond))
simulation.context.setPositions(pdb_file.positions)
simulation.minimizeEnergy()

Run a few steps to make sure it's working

In [8]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

## Run the NanoVer Server

We'll use NanoVer's `OmniRunner` to simplify running the server, passing the simulation as an `ASESimulation`. For a more detailed walkthrough of this process, check out our [neuraminidase example](./ase_openmm_neuraminidase.ipynb).

In [9]:
# Import ASE integrator and units
from ase.md import Langevin
import ase.units as ase_units

# Import the relevant NanoVer classes
from nanover.omni import OmniRunner
from nanover.omni.ase import ASESimulation
from nanover.ase.omm_calculator import OpenMMCalculator

# Define the calculator using the OpenMM simulation defined above
calculator = OpenMMCalculator(simulation)

# Define the atoms object and set it's calculator as the OpenMMCalculator
atoms = calculator.generate_atoms()
atoms.calc = calculator

# Define the integrator for the simulation
dynamics = Langevin(atoms, timestep=1.0 * ase_units.fs, temperature_K=300, friction=1.0e-03)

# Define the ASESimulation
graphene_ase_omm_sim = ASESimulation.from_ase_dynamics(dynamics, ase_atoms_to_frame_data=calculator.make_frame_converter())

# Pass the simulation to the OmniRunner, load the simulation and pause it
imd_runner = OmniRunner.with_basic_server(graphene_ase_omm_sim, name="graphene-ase-omm-server", port=0)
imd_runner.next()
imd_runner.pause()

RUNNING TEMP 200


Let's run a few steps and check that everything is working as expected.

In [10]:
graphene_ase_omm_sim.dynamics.run(10)

True

In [11]:
graphene_ase_omm_sim.dynamics.get_number_of_steps()

10

Let's check the potential energy too.

In [12]:
graphene_ase_omm_sim.atoms.get_potential_energy()

22.73017453475752

Now we're ready to go! Let's start the simulation and leave it to run in the background.

In [13]:
# Start running the simulation
imd_runner.play()

We can print the details of the server by running the following cell:

In [14]:
imd_runner.print_basic_info()

graphene-ase-omm-server: serving on [::]:57795


## Controlling the Physics From the Notebook

Since we're running the simulation with ASE, we can change the parameters while it's running. 
The cell below sets up some methods for changing the temperature, friction and timestep

In [15]:
temp_min_val = 0
temp_max_val = 10000
friction_min_val = 0.01
friction_max_val = 100
timestep_min_val = 0.01
timestep_max_val = 1.5
import ase.units as units

def set_temperature(temperature=300):
    """
    Sets the temperature in the ASE simulation.

    :param temperature: Temperature to set, in kelvin.
    """
    print("RUNNING TEMP", temperature)
    if not temp_min_val <= temperature <= temp_max_val:
        raise ValueError(f'Temperature must be in range {temp_min_val} - {temp_max_val} Kelvin.')
    graphene_ase_omm_sim.dynamics.set_temperature(temperature_K=temperature)


def set_friction(friction=1):
    """
    Sets the friction in the ASE simulation.

    :param friction: Friction, in ASE units * 1000, for visualisation purposes
    """

    if not friction_min_val <= friction <= friction_max_val:
        raise ValueError(f'Friction must be in range {friction_min_val} - {friction_max_val}.')
    graphene_ase_omm_sim.dynamics.set_friction(friction / 1000.0)


def set_timestep(timestep=0.5):
    """
    Sets the timestep in the ASE simulation.

    :param timestep: Timestep, in femtoseconds.
    """

    if not timestep_min_val <= timestep <= timestep_max_val:
        raise ValueError(f'Timestep must be in range {timestep_min_val} - {timestep_max_val}')
    timestep = timestep * units.fs
    graphene_ase_omm_sim.dynamics.set_timestep(timestep)

Now we set up some sliders and buttons so we can adjust these on the fly in the notebook

In [16]:
# imports for sliders
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

In [17]:
# Sliders for temperature, friction and timestep
interact(set_temperature, temperature=(temp_min_val,temp_max_val));
interact(set_friction, friction=(friction_min_val,friction_max_val, 1.0));
interact(set_timestep, timestep=(timestep_min_val,timestep_max_val, 0.01));

# buttons and toggles for playing and reset
reset_button = widgets.Button(description="Restart Simulation")
play_button = widgets.ToggleButton(description="Playing")
output = widgets.Output()
display(reset_button, output)
display(play_button, output)

def on_reset_clicked(b):
    with output:
        imd_runner.reset()

def on_play_clicked(obj):
    with output:
        if obj['new']:  
            imd_runner.play()
        else:
            imd_runner.pause()

reset_button.on_click(on_reset_clicked)
play_button.observe(on_play_clicked, 'value')

interactive(children=(IntSlider(value=300, description='temperature', max=10000), Output()), _dom_classes=('wi…

interactive(children=(FloatSlider(value=1.0, description='friction', min=0.01, step=1.0), Output()), _dom_clas…

interactive(children=(FloatSlider(value=0.5, description='timestep', max=1.5, min=0.01, step=0.01), Output()),…

Button(description='Restart Simulation', style=ButtonStyle())

Output()

ToggleButton(value=False, description='Playing')

Output()

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

from nanover.utilities.change_buffers import DictionaryChange

def reset_molecule():
    imd_runner.reset()

def set_box_locked(locked=False):
    imd_runner.app_server.update_state(None, DictionaryChange({ "suggested.box.locked": locked }))

def set_box_hidden(hidden=False):
    imd_runner.app_server.update_state(None, DictionaryChange({ "suggested.box.hidden": hidden }))

def set_force_type(type: str):
    imd_runner.app_server.update_state(None, DictionaryChange({ "suggested.interaction.type": type }))

def set_force_scale(scale: int):
    imd_runner.app_server.update_state(None, DictionaryChange({ "suggested.interaction.scale": scale }))

def set_passthrough(passthrough: float):
    imd_runner.app_server.update_state(None, DictionaryChange({ "suggested.passthrough": passthrough }))

reset_button = widgets.Button(description="Reset Molecule")
reset_button.on_click(reset_molecule)

display(reset_button)
interact(set_box_locked, locked=widgets.ToggleButton(description='Lock Box'))
interact(set_box_hidden, hidden=widgets.ToggleButton(description='Hide Box'))
interact(set_force_type, type=widgets.Dropdown(options=[("Gaussian", "gaussian"), ("Harmonic", "harmonic"), ("Constant", "constant")], description="Force Type"))
interact(set_force_scale, scale=widgets.IntSlider(description="Force Scale", min=1, max=1000))
interact(set_passthrough, passthrough=widgets.FloatSlider(description="Passthrough", min=0, max=1))

Enter the server in VR and see how the dynamics change when you lower the temperature and and massively increase the friction!

## Remote Control Commands

While controlling these parameters from the notebook is pretty cool, doing it from VR or a dedicated application would be even better. 

NanoVer provides a mechanism for doing this via *commands*. A command consists of a command name and a handler function to call when the client requests to run a command by that name.

Let's set up our timestep, friction and temperature methods as commands

In [19]:
# Methods for interacting with the simulation.
TIMESTEP_COMMAND = "sim/timestep"
FRICTION_COMMAND = "sim/friction"
TEMPERATURE_COMMAND = "sim/temperature"

# the following line unregisters the commands if they've already been registered. 
for command in [TIMESTEP_COMMAND, FRICTION_COMMAND, TEMPERATURE_COMMAND]:
    try:
        imd_runner.app_server.unregister_command(command)
    except:
        pass

imd_runner.app_server.register_command(TIMESTEP_COMMAND, set_timestep)
imd_runner.app_server.register_command(TEMPERATURE_COMMAND, set_temperature)
imd_runner.app_server.register_command(FRICTION_COMMAND, set_friction)

Now, we can connect a client, and call the commands

In [20]:
from nanover.websocket import NanoverImdClient
client =  NanoverImdClient.from_runner(imd_runner)

We can see all the available commands, note that play, pause, reset and step are already registered, as are the ones we've just added 

In [21]:
commands = client.update_available_commands()
dict(commands).keys()

dict_keys(['commands/list', 'multiuser/radially-orient-origins', 'playback/load', 'playback/next', 'playback/list', 'playback/reset', 'playback/pause', 'playback/play', 'playback/step', 'sim/timestep', 'sim/temperature', 'sim/friction'])

So now we can set the temperature remotely (try this on another computer!):

In [22]:
client.run_command_blocking('sim/temperature', temperature=200)
# print out the temperature to check it's worked, we have to convert from ASE units to Kelvin
graphene_ase_omm_sim.dynamics.temp / units.kB

199.99999999999997

## Gracefully Terminate

In [23]:
client.close()
imd_runner.close()

## Next Steps

* Explore setting up [commands and synchronizing state](../fundamentals/commands_and_state.ipynb)